In [1]:
import torch
import copy
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import optim
from torch import nn
from torch.nn import functional as F

from collections import OrderedDict

import argparse
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
train_dataset = datasets.MNIST(root='C:/Users/WorkStation/Desktop/Git_Repositories/Dataset', train = True, transform = transforms.ToTensor(), download = True)
test_dataset = datasets.MNIST(root='C:/Users/WorkStation/Desktop/Git_Repositories/Dataset', train = False, transform = transforms.ToTensor(), download = True)

C:\Users\WorkStation\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


##### **Functions(for all)**

In [3]:
def load_data(args):    
    ldt = pd.read_csv(f"FL(mining_data)/parameter_data/param_data_{args.num_clients}.csv")
    ldt = ldt.values
    ldt = np.delete(ldt, 0, 1)
    return ldt

def data_split(train_dataset, args):
    dataset_list=[]

    num_data_arr = np.ones((args.num_clients,), dtype=int) * int(len(train_dataset)/args.num_clients)
    dataset_list = torch.utils.data.random_split(train_dataset, num_data_arr)
    return dataset_list

def list_np_tensor(param_data):
    data = np.concatenate((param_data['net.0.weight'].cpu().numpy(),np.expand_dims(param_data['net.0.bias'].cpu().numpy(), axis=1)),axis=1)
    data = torch.Tensor(data)
    return data

def plot_loss(FL_round_loss, FL_round_acc, round_losses, round_acces, avg_round_losses, avg_round_acces):
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1,2,1)
    plt.plot(FL_round_loss, label = 'FL')
    plt.plot(round_losses, label = 'FL+AE')
    plt.plot(avg_round_losses, label = 'FL+AE(w_avg)')
    plt.xlabel('Rounds')
    plt.grid()
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(FL_round_acc, label = 'FL')
    plt.plot(round_acces, label = 'FL+AE')
    plt.plot(avg_round_acces, label = 'FL+AE(w_avg)')
    plt.xlabel('Rounds')
    plt.grid()
    plt.legend()

    plt.show()

##### **Model**

In [4]:
class Encoder(nn.Module):
    def __init__(self, hidden_dim):
        super(Encoder, self).__init__()

        self.hidden_dim = hidden_dim
        self.encoder = nn.Sequential(nn.Linear(7851, 785),
                                    nn.ReLU(),
                                    nn.Linear(785, 50),
                                    nn.ReLU(),
                                    nn.Linear(50, self.hidden_dim)
                                    )
            
    def forward(self, x):
        output = self.encoder(x)
        return output


class Decoder(nn.Module):
    def __init__(self, hidden_dim):
        super(Decoder, self).__init__()

        self.hidden_dim = hidden_dim
        self.decoder = nn.Sequential(nn.Linear(self.hidden_dim, 50),
                                    nn.ReLU(),
                                    nn.Linear(50, 785),
                                    nn.ReLU(),
                                    nn.Linear(785, 7851)
                                    )
            
    def forward(self, x):
        output = self.decoder(x)
        return output
        

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.net = nn.Sequential(nn.Linear(28*28, 10)
                                )

    def forward(self, x):
        x = x.view(-1,28*28)
        out = self.net(x)
        return out

##### **FL Functions**

In [5]:
def FL_define_client(partition, r, num, args):
    train_loader = DataLoader(partition['train'][num], batch_size=args.batch_size, shuffle=True)
    
    if r == 0:
        net = Net().to(device)
    else:
        net = torch.load(args.INIT_PATH)
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=args.lr)

    return net, train_loader, criterion, optimizer

def FL_train(net, train_loader, criterion, optimizer, args):

    for epoch in range(args.num_epochs):        
        for images, labels in iter(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad() 
            
            outputs = net(images)  
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

    torch.save(net, args.NET_PATH)
    return net

def FL_Fed_AVG(model, client_model_list, args):
    model.train()
    local_weights = []

    for i in range(args.num_clients):
        local_weights.append(copy.deepcopy(client_model_list[i].state_dict()))
    w_avg = copy.deepcopy(local_weights[0]) #type of w_avg : <class 'collections.OrderedDict'>

    for key in w_avg.keys():
        for i in range(1,len(local_weights)):
            w_avg[key] += local_weights[i][key]

        w_avg[key] = torch.div(w_avg[key], len(local_weights))

    global_weights = w_avg
    model.load_state_dict(global_weights)
    torch.save(model, args.INIT_PATH)

    return model

def FL_test(model, partition, criterion, args):
    test_loader = DataLoader(partition['test'], batch_size=args.batch_size, shuffle=True)
    model.eval()
        
    correct = 0 # 정답을 맞힌 이미지 개수
    total = 0 # 전체 이미지 개수
    loss = 0
    round_accuracy = 0
    round_loss = 0

    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()
        total += targets.size(0)
    
    round_loss = loss / len(test_loader)
    round_accuracy = (correct / total) * 100.0
    
    return round_loss, round_accuracy

In [6]:
def FL_experiment(partition, args):
    model = Net().to(device)
    torch.save(model, args.INIT_PATH)

    round_losses = []
    round_acces = []

    for r in range(args.num_rounds):
        client_model_list = []
        
        for num in range(args.num_clients):
            setattr(args, 'NET_PATH', f'FL_model/client = {args.num_clients}/client_{num}.pth')
            
            net, train_loader, criterion, optimizer = FL_define_client(partition, r, num, args)
            net = FL_train(net, train_loader, criterion, optimizer, args)
            client_model_list.append(net)

        model = FL_Fed_AVG(model, client_model_list, args)
        round_loss, round_acc = FL_test(model, partition, criterion, args)

        if r%50 == 0 or r==(args.num_rounds-1):
            print(f'[Round {r}]  Round Loss: {round_loss}  Round Acc: {round_acc:.2f}')

        round_losses.append(round_loss)
        round_acces.append(round_acc)
        
    return round_losses, round_acces

##### **FL+AE Functions**

In [7]:
def define_client(partition, global_weights, r, num, args):
    train_loader = DataLoader(partition['train'][num], batch_size=args.batch_size, shuffle=True)
    
    if r == 0:
        net = Net().to(device)

    else:
        net = Net().to(device)        
        net.load_state_dict(global_weights)
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=args.lr)

    return net, train_loader, criterion, optimizer

def train(net, train_loader, criterion, optimizer, args):
    net.train()

    for epoch in range(args.num_epochs):        
        for images, labels in iter(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad() 
            
            outputs = net(images)  
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
    return net

def client_encode(net, r, encoder):
    round_tensor = torch.tensor(r).view(-1, 1).to(device)

    param_data = copy.deepcopy(net.state_dict())
    tensor_data = list_np_tensor(param_data).view(-1,7850).to(device)
    dt_r = torch.cat((tensor_data, round_tensor), dim=1)
    encoded_data = encoder(dt_r)
    return encoded_data

def Dec_Fed_AVG(model, decoder, client_enc_data_list, args):
    model.train()
    decoded_data_list = []
    global_weights = OrderedDict()

    for i in range(args.num_clients):
        client_enc_data_list[i] = client_enc_data_list[i].to(device)
        out = decoder(client_enc_data_list[i])[:,:-1].view((-1, 785)).cpu().detach().numpy()    
        decoded_data_list.append(out)

    w_avg = np.mean(decoded_data_list, axis=0)
    global_weights['net.0.weight'] = torch.tensor(w_avg[:,:784]).clone().detach()
    global_weights['net.0.bias'] = torch.tensor(w_avg[:,784:].squeeze()).clone().detach()

    model.load_state_dict(global_weights)

    return model, global_weights

def test(model, partition, criterion, args):
    test_loader = DataLoader(partition['test'], batch_size=args.batch_size, shuffle=True)
    model.eval()
        
    correct = 0 
    total = 0 
    loss = 0
    round_accuracy = 0
    round_loss = 0

    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()
        total += targets.size(0)
    
    round_loss = loss / len(test_loader)
    round_accuracy = (correct / total) * 100.0
    
    return round_loss, round_accuracy

In [8]:
def experiment(partition, args):
    encoder = Encoder(args.hidden_dim)
    decoder = Decoder(args.hidden_dim)
    model = Net()
    encoder.load_state_dict(torch.load(args.enc_path))
    decoder.load_state_dict(torch.load(args.dec_path))

    encoder, decoder, model = encoder.to(device), decoder.to(device), model.to(device)
    
    global_weights = OrderedDict()
    round_losses = []
    round_acces = []


    for r in range(args.num_rounds):
        client_enc_data_list = []

        for num in range(args.num_clients):
            net, train_loader, criterion, optimizer = define_client(partition, global_weights, r, num, args)
            net = train(net, train_loader, criterion, optimizer, args)
            client_enc_data = client_encode(net, r, encoder)
            client_enc_data_list.append(client_enc_data)

        model, global_weights = Dec_Fed_AVG(model, decoder, client_enc_data_list, args)
        round_loss, round_acc = test(model, partition, criterion, args)

        if r%50 == 0 or r == (args.num_rounds-1):
            print(f'[Round: {r}]  Round Loss: {round_loss}  Round Acc: {round_acc:.2f}')

        round_losses.append(round_loss)
        round_acces.append(round_acc)
        
    return round_losses, round_acces

In [9]:
def save_list(loss, acc, num_data=None, avg=False):
    dtf_loss, dtf_acc = pd.DataFrame(loss), pd.DataFrame(acc)

    if num_data == None:
        dtf_loss.to_csv(f"paper_data/[loss]Normal_FL.csv")
        dtf_acc.to_csv(f"paper_data/[acc]Normal_FL.csv")
        
    else:
        if avg==False:
            dtf_loss.to_csv(f"paper_data/[loss]Non_Avg_FL_{num_data}.csv")
            dtf_acc.to_csv(f"paper_data/[acc]Non_Avg_FL_{num_data}.csv")
        else:
            dtf_loss.to_csv(f"paper_data/[loss]Avg_FL_{num_data}.csv")
            dtf_acc.to_csv(f"paper_data/[acc]Avg_FL_{num_data}.csv")
    print('save loss & acc')

##### **Normal FL**

In [10]:
seed = 1228
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

args.exp_name = 'final'

args.num_epochs = 5
args.num_rounds = 500
args.num_clients = 3
args.lr = 0.0001

args.batch_size = 100

args.NET_PATH = ''
args.INIT_PATH = f'FL_model/client = {args.num_clients}/initialize_weight.pth'

train_set_list = data_split(train_dataset, args)
partition = {'train':train_set_list, 'test':test_dataset}

FL_round_loss, FL_round_acc = FL_experiment(partition, args)
save_list(FL_round_loss, FL_round_acc)

[Round 0]  Round Loss: 0.7766053175926209  Round Acc: 84.96
[Round 50]  Round Loss: 0.2635316574573517  Round Acc: 92.67
[Round 100]  Round Loss: 0.2630053900927305  Round Acc: 92.82
[Round 150]  Round Loss: 0.2662125315517187  Round Acc: 92.80
[Round 200]  Round Loss: 0.2698613403737545  Round Acc: 92.92
[Round 250]  Round Loss: 0.27387715376913546  Round Acc: 92.84
[Round 300]  Round Loss: 0.27760546617209914  Round Acc: 92.89
[Round 350]  Round Loss: 0.2811790708452463  Round Acc: 92.89
[Round 400]  Round Loss: 0.28480591617524625  Round Acc: 92.90
[Round 450]  Round Loss: 0.2881507502496243  Round Acc: 92.91
[Round 499]  Round Loss: 0.29114479281008243  Round Acc: 92.88
save loss & acc


##### **[FL+AE] Non_Avg vs Avg**

In [12]:
seed = 1228
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

# ====== Model Capacity ====== #
args.num_epochs = 5
args.num_rounds = 500
args.num_clients = 3
args.lr = 0.0001

args.batch_size = 100
args.hidden_dim = 15
args.num_data = 100

args.avg = False

name_var1 = 'num_data'
name_var2 = 'avg'

list_var1 = [50, 500]
list_var2 = [False, True]


for var1 in list_var1:
    print(f"===== {name_var1}: {var1*5} =====")
    setattr(args, name_var1, var1)

    for var2 in list_var2:
        print(f"{name_var2}: {var2}") 
        setattr(args, name_var2, var2)
  
        train_set_list = data_split(train_dataset, args)
        partition = {'train':train_set_list, 'test':test_dataset}

        if var2 == False:
            args.enc_path = f"AE_model/client = {args.num_clients}/enc_num_data_{var1}.pth"
            args.dec_path = f"AE_model/client = {args.num_clients}/dec_num_data_{var1}.pth"
            round_losses, round_acces = experiment(partition, args)
            save_list(round_losses, round_acces, num_data=var1, avg=var2)

        else:
            args.enc_path = f"FL(mining_data)/w_avg_AE_model/client = {args.num_clients}/enc_num_data_{var1}.pth"
            args.dec_path = f"FL(mining_data)/w_avg_AE_model/client = {args.num_clients}/dec_num_data_{var1}.pth"
            avg_round_losses, avg_round_acces = experiment(partition, args)
            save_list(avg_round_losses, avg_round_acces, num_data=var1, avg=var2)

    plot_loss(FL_round_loss, FL_round_acc, round_losses, round_acces, avg_round_losses, avg_round_acces)

===== num_data: 250 =====
avg: False
[Round: 0]  Round Loss: 0.6027184376120567  Round Acc: 86.74
[Round: 50]  Round Loss: 0.3913286290317774  Round Acc: 91.82
[Round: 100]  Round Loss: 0.8110113221406937  Round Acc: 90.89


In [ ]:
def load_data(num_data=None, avg=False): 
    if num_data == None:
        ldt_loss = pd.read_csv(f"paper_data/[loss]Normal_FL.csv")
        ldt_acc = pd.read_csv(f"paper_data/[acc]Normal_FL.csv")
        
    else:
        if avg==False:
            ldt_loss = pd.read_csv(f"paper_data/[loss]Non_Avg_FL_{num_data}.csv")
            ldt_acc = pd.read_csv(f"paper_data/[acc]Non_Avg_FL_{num_data}.csv")

        else:
            ldt_loss = pd.read_csv(f"paper_data/[loss]Avg_FL_{num_data}.csv")
            ldt_acc = pd.read_csv(f"paper_data/[acc]Avg_FL_{num_data}.csv")

    ldt_loss, ldt_acc = np.delete(ldt_loss.values, 0, 1), np.delete(ldt_acc.values, 0, 1)
    return ldt_loss, ldt_acc

In [ ]:
Normal_FL_loss, Normal_FL_acc = load_data()
print(Normal_FL_loss, Normal_FL_acc)

for var1 in list_var1:
    Non_Avg_loss, Non_Avg_acc = load_data(num_data=var1, avg=False)
    Avg_loss, Avg_acc = load_data(num_data=var1, avg=True)
    print(Non_Avg_loss, Non_Avg_acc)
    print(Avg_loss, Avg_acc)
    print('\n')

[[0.34349495]
 [0.30909618]
 [0.31210935]
 [0.31298341]
 [0.31353626]] [[91.5 ]
 [92.65]
 [92.63]
 [92.63]
 [92.62]]
[[0.34349495]
 [0.30909618]
 [0.31210935]
 [0.31298341]
 [0.31353626]] [[91.5 ]
 [92.65]
 [92.63]
 [92.63]
 [92.62]]


[[0.29874221]
 [0.28698858]
 [0.28863793]
 [0.28930337]
 [0.28981996]] [[91.92]
 [92.72]
 [92.68]
 [92.67]
 [92.67]]
[[0.29874221]
 [0.28698858]
 [0.28863793]
 [0.28930337]
 [0.28981996]] [[91.92]
 [92.72]
 [92.68]
 [92.67]
 [92.67]]


